In [2]:
import pybamm

parameter_values = pybamm.ParameterValues.create_from_bpx(r"C:\Users\MVeerasi.ANALOG\Documents\InternshipFY24\Li-Ion-Battery-Simulator\BatterySimulator\Simulation\LFP\lfp_18650_cell_BPX.json")
parameter_values

FileNotFoundError: [Errno 2] No such file or directory: 'Li-Ion-Battery-Simulator\\BatterySimulator\\Simulation\\LFP\\lfp_18650_cell_BPX.json'

In [21]:
# could implement a way to search what parameter you want to edit.
parameter_values.search("voltage")


Lower voltage cut-off [V]	2.0
Open-circuit voltage at 0% SOC [V]	2.0
Open-circuit voltage at 100% SOC [V]	3.65
Upper voltage cut-off [V]	3.65


In [37]:
# adjusting
parameter_values["Current function [A]"] = 10
parameter_values["Open-circuit voltage at 100% SOC [V]"] = 4.2

In [38]:
# Create a DFN model
model = pybamm.lithium_ion.DFN()

In [39]:
# solver
solver = pybamm.CasadiSolver("safe", atol=1e-6, rtol=1e-6)

# Prevent solver failure if interpolant bounds are exceeded by a negligible amount
solver._on_extrapolation = "warn"

In [40]:
# Define an experiment
experiment = pybamm.Experiment(
    [
        (
            "Discharge at C/5 for 10 hours or until 2.5 V",
            "Rest for 1 hour",
            "Charge at 1 A until 3.5 V",
            "Hold at 3.5 V until 10 mA",
            "Rest for 1 hour",
        ),
    ]
    * 2
)

In [41]:
# Create a simulation object
sim = pybamm.Simulation(model, parameter_values=parameter_values, solver=solver, experiment=experiment)

In [42]:
# Simulate
sim.solve()

At t = 238.258 and h = 5.10856e-13, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 238.258 and h = 1.11737e-18, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 118.258 and h = 7.54588e-18, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 233.714 and h = 2.1527e-07, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 236.506 and h = 1.21426e-09, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 135.305 and h = 1.82393e-13, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 135.305 and h = 1.0935e-13, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 257.867 and h = 3.58477e-09, the corrector convergence failed repeatedly or with |h| = hmin.


In [43]:
# Plot results
sim.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=19.093923303580432, step=0.19093923303580432…

In [8]:
NMC=(r"C:\Users\markv\Documents\Repos\2024\BatterySimulator_v2\BatterySimulator_SimulationBackend\Li-Ion-Battery-Simulator\BatterySimulator\Simulation\NMC\AE_gen1_BPX.json")
LFP=(r"C:\Users\MVeerasi.ANALOG\Documents\InternshipFY24\Li-Ion-Battery-Simulator\BatterySimulator\Simulation\LFP\lfp_18650_cell_BPX.json")

# OOP SUCKS

In [14]:
import pybamm

class SetupSimulation:
    def __init__(self, chemistry, electrochemical_model, solver) -> None:
        self.chemistry = chemistry
        self.adjusted_parameters = {} # Placeholder dictionary for user-adjusted parameters.
        self.electrochemical_model = electrochemical_model # electrochemistry-based lithium-ion battery model 
        self.solver = solver 
        # could put cell_type to help filter dupe chemistries?

    ADJUSTABLE_PARAMETERS = [
            "Ambient temperature [K]",
            "Initial temperature [K]",
            "Lower voltage cut-off [V]",
            "Upper voltage cut-off [V]",
            "Current function [A]",
        ]
    
    # This would be something like localhost:8080/chemistry=LFP/
    def set_battery_chemistry(self, chemistry):
        if chemistry == "LFP":
            parameter_values = pybamm.ParameterValues.create_from_bpx(LFP)
            # parameter_values = pybamm.ParameterValues("Chen2020")
            return parameter_values
        elif chemistry == "NMC":
            parameter_values = pybamm.ParameterValues.create_from_bpx(NMC)
            # parameter_values = pybamm.ParameterValues("Chen2020")
            return parameter_values
        
    # This would be something like localhost:8080/chemistry=LFP/model=dfn
    def set_electrochemical_model(self, electrochemical_model="DFN"): # default will be DFN, idk what it does and I will worry about this when it may matter - maybe
        if electrochemical_model == "DFN":
            return pybamm.lithium_ion.DFN()
        elif electrochemical_model == "SPM":
            return pybamm.lithium_ion.SPM()
        elif electrochemical_model == "SPMe":
            return pybamm.lithium_ion.SPMe()
        else:
            raise ValueError(f"Unknown model type: {electrochemical_model} Please pick DFN, SPM or SPMe")
        
    # # This would be something like localhost:8080/chemistry=LFP/model=dfn/adjustedParameters=true
    # def set_adjusted_parameter_values(self, adjusted_parameters):
    #     parameter_values.update(adjusted_parameters)
    
    # # This would be something like localhost:8080/chemistry=LFP/model=dfn/adjustedParameters=true/solver=casadi
    def set_solver(self, solver):
        solver = pybamm.CasadiSolver("safe", atol=1e-6, rtol=1e-6)
        # Prevent solver failure if interpolant bounds are exceeded by a negligible amount
        solver._on_extrapolation = "warn"
        return solver

# from SetupSimulation import SetupSimulation - not needed in notebook
class SimulateBattery:
    def __init__(self, chemistry, electrochemical_model, solver):
        self.chemistry = chemistry
        self.electrochemical_model = electrochemical_model
        self.solver = solver
        self.setup_simulation = SetupSimulation(chemistry, electrochemical_model, solver)

    def simulate_battery_base_model(self):
        parameter_values = self.setup_simulation.set_battery_chemistry(self.chemistry)
        model = self.setup_simulation.set_electrochemical_model(self.electrochemical_model)
        solver = self.setup_simulation.set_solver(self.solver)

        simulation = pybamm.Simulation(model, parameter_values=parameter_values, solver=solver)

        # Example: Simulating for a specific time range
        time_span = [0, 3700]  # Adjust this according to your needs
        solution = simulation.solve(time_span)

        return solution

if __name__ == "__main__":
    sim = SimulateBattery(chemistry="LFP", electrochemical_model="DFN", solver="CasadiSolver")
    simulation = sim.simulate_battery_base_model()
    
    simulation.plot()

The linesearch algorithm failed with too small a step.
The linesearch algorithm failed with too small a step.
At t = 452.524 and h = 1.30213e-18, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 153.535 and h = 2.52947e-17, the corrector convergence failed repeatedly or with |h| = hmin.


interactive(children=(FloatSlider(value=0.0, description='t', max=3579.1633260284893, step=35.7916332602849), …

In [47]:
import pybamm
from pydantic import BaseModel, StrictStr, validator

class SetupModel(BaseModel):
    chemistry: StrictStr 
    electrochemical_model: StrictStr
    solver: StrictStr

    # make a validator to validate if the the chemistry is not in the config then cant work
    @validator("chemistry")
    def validate_chemistry(cls, chemistry_type):
        if chemistry_type not in {"LFP", "NMC"}: # update this condition so if not in the battery.config
            raise ValueError(f"Invalid chemistry Model Type: {chemistry_type}")

    @validator("electrochemical_model")
    def validate_electrochemical_model(cls, model_type):
        if model_type not in {"DFN", "SPM", "SPMe"}:
            raise ValueError(f"Invalid Electrochemical Model Type: {model_type}")
        
    # might make solver it's own class because of it's solving type

# example model setup where we will use the about:energy LFP model through BPX
setupModel = SetupModel(
    chemistry="LFP",
    electrochemical_model="DFN",
    solver="CasadiSolver"
)

def set_solver(solver):
    solver = pybamm.CasadiSolver("safe", atol=1e-6, rtol=1e-6)
    # Prevent solver failure if interpolant bounds are exceeded by a negligible amount
    solver._on_extrapolation = "warn"
    return solver


    

ValidationError: 1 validation error for User
account_id
  value is not a valid integer (type=type_error.integer)